# Freq-E example usage 
Starting with a collection of text documents. This notebook walks through: 

1. Getting text in a format suitable for input into freq-e

2. How to run freq-e to obtain prevalence estimates on the test set. 

In [1]:
from __future__ import division, print_function
import numpy as np 
import json
from sklearn.feature_extraction import DictVectorizer

In [4]:
import estimate
%load_ext autoreload
%autoreload 2

In [6]:
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

## Pre-processing 
We will use the Yelp academic dataset as an example. The text representation will be unigram counts (e.g. "bag-of-words"). Here, we have already calcuated the BOW counts and saved them as a .json file. The y-values are negative sentiment (y=0) and positive sentiment (y=1). 

In [7]:
def load_x_y_from_json(file_name): 
    count_dicts = []
    y = []
    for line in open(file_name): 
        dd = json.loads(line)
        counts = dd['counts'].copy()
        cc = dd['class']
        count_dicts.append(counts); y.append(cc)
    return count_dicts, np.array(y)  

In [8]:
def prune_vocab(X_train, dv_vocab): 
    #remove words that occur in <5 docs 
    xx=X_train.copy()
    xx[xx>0]=1
    w_df = np.asarray(xx.sum(0)).flatten()
    new_vocab_mask = w_df >= 5
    print("Orig vocab %d, pruned %d" % (len(w_df), np.sum(new_vocab_mask)))
    X_train = X_train[:,new_vocab_mask]
    dv_vocab = dv_vocab[new_vocab_mask]
    return X_train, dv_vocab, new_vocab_mask

In [9]:
#get train data 
dv = DictVectorizer()
train_count_dicts, y_train = load_x_y_from_json('example_data/train_yelp.json')
X_train = dv.fit_transform(train_count_dicts).toarray()
dv_vocab = np.array(dv.feature_names_)
X_train, dv_vocab, new_vocab_mask = prune_vocab(X_train, dv_vocab)
print(type(X_train), type(y_train))
print(X_train.shape, y_train.shape)
assert X_train.shape[0] == y_train.shape[0]

Orig vocab 14791, pruned 3112
<class 'numpy.ndarray'> <class 'numpy.ndarray'>
(2000, 3112) (2000,)


In [10]:
def transform_test(test_count_dicts, new_vocab_mask): 
    X_test = dv.transform(test_count_dicts).toarray()
    X_test = X_test[:,new_vocab_mask]
    return X_test

In [11]:
# get test data (1 test group) 
# NOTE: the test group is the "inference" group in a real-word setting
# here we have labels on the test set, but in a real-word setting there 
# would most likely not be labels on the test set
test_count_dicts, y_test = load_x_y_from_json('example_data/test_yelp.json')
X_test = transform_test(test_count_dicts, new_vocab_mask)
print(type(X_test), type(y_test))
print(X_test.shape, y_test.shape)
assert X_test.shape[1] == X_train.shape[1]

<class 'numpy.ndarray'> <class 'numpy.ndarray'>
(2000, 3112) (2000,)


# Freq-e usage

## Inference
Freq-e inference will return (1) a point estimate of the class frequency/proportions and (2) a confidence interval for the point estimate. 

There are two distict methods one can get a frequency estimate: 
1. Pass in a trainined scikit-learn linear model (such as logistic regtression)
2. Pass in the predicted probabilities of the positive class of the test set. 

You might want to use Method 2 in the case where you have some specialzed classifier that is not a part of sklearn.  

## Method 1 (pass in instance of sklearn.linear_model)

### Training 
- In order to select the best discriminitive classifier, we will do a grid search over the L1 penalties for LogReg, evaluating on cross-entropy over 10 cross-validation folds 

In [14]:
freq_e = estimate.FreqEstimate()
trained_model = freq_e.fit_disc_classifier(X_train, y_train)

TRAINING DISCRIMINATIVE MODEL
LogisticRegression(C=0.5, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l1', random_state=None, solver='liblinear',
          tol=0.0001, verbose=0, warm_start=False)
Training mean accuracy= 0.9635


#### Large test group

In [15]:
# here we can know the true proportions because we have access to the test labels 
print('TRUE')
print(np.mean(y_test))

TRUE
0.769


In [16]:
#naive method = PCC (probabilistic classify and count)
print('PCC')
print(np.mean(trained_model.predict_proba(X_test)[:, 1]))

PCC
0.7694202722821978


In [19]:
print('FREQ-E ESTIMATED')
out = freq_e.infer_freq(y_train, X_test, conf_level=0.95, trained_model=trained_model, test_pred_probs=None)
print(out)

FREQ-E ESTIMATED
{'point': 0.772, 'conf_interval': (0.748, 0.795)}


#### High prevalence test group  

In [20]:
test_count_dicts2, y_test2 = load_x_y_from_json('testgroup_zpoZ6WyQUYff18-z4ZU1mA/zpoZ6WyQUYff18-z4ZU1mA')
X_test2 = transform_test(test_count_dicts2, new_vocab_mask)
print(X_test2.shape, y_test2.shape)

(415, 3112) (415,)


In [21]:
print('TRUE')
print(np.mean(y_test2))

TRUE
0.9686746987951808


In [22]:
print('PCC')
print(np.mean(trained_model.predict_proba(X_test2)[:, 1]))

PCC
0.9019348999589916


In [23]:
print('FREQ-E ESTIMATED')
out = freq_e.infer_freq(y_train, X_test2, conf_level=0.95, trained_model=trained_model, test_pred_probs=None)
print(out)

FREQ-E ESTIMATED
{'point': 0.973, 'conf_interval': (0.9440000000000001, 0.992)}


#### Low prevalence test group 

In [24]:
test_count_dicts3, y_test3 = load_x_y_from_json('dWFUKB_HPBIE87AFBHEb_w')
X_test3 = transform_test(test_count_dicts3, new_vocab_mask)
print(X_test3.shape, y_test3.shape)

(825, 3112) (825,)


In [25]:
print('TRUE')
print(np.mean(y_test3))

TRUE
0.13696969696969696


In [26]:
print('PCC')
print(np.mean(trained_model.predict_proba(X_test3)[:, 1]))

PCC
0.31147433158714605


In [27]:
print('FREQ-E ESTIMATED')
out = freq_e.infer_freq(y_train, X_test3, conf_level=0.95, trained_model=trained_model, test_pred_probs=None)
print(out)

FREQ-E ESTIMATED
{'point': 0.046, 'conf_interval': (0.028, 0.069)}


## Method 2 (pass in predicted probabilites on the test set)
We will show this results in the same frequency estimate as the last group. 

In [32]:
# get the probabilities for the positive class 
test_pred_probs = trained_model.predict_proba(X_test3)[:, 1]
print(test_pred_probs[0:5])

[2.92084161e-08 8.55934549e-01 8.43841482e-01 6.97602815e-02
 1.48921023e-01]


In [36]:
print('FREQ-E ESTIMATED')
out = freq_e.infer_freq(y_train, X_test3, conf_level=0.95, trained_model=None, test_pred_probs=test_pred_probs)
print(out)

FREQ-E ESTIMATED
{'point': 0.046, 'conf_interval': (0.028, 0.069)}
